<a href="https://colab.research.google.com/github/Alebraco/phagetool/blob/main/Phage_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bio
from Bio import Entrez,SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.5 MB/s eta 0:00:00


In [ ]:
Entrez.email = 'alekey039@hotmail.com'

In [ ]:
handle = Entrez.einfo()
info = Entrez.read(handle)
print(info)

{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}


In [ ]:
def retrieve_ids(max, query):
  ids = []
  start = 0
  while(True):
    try:
      handle = Entrez.esearch(db = 'ipg', retmax = max, retstart = start, term = query)
      rec = Entrez.read(handle)
      handle.close()

    except Exception as error:
      print('Search failed, trying again in 1 second:', error)
      time.sleep(1)
      continue

    if len(rec['IdList']) == 0:
      break

    start += max
    ids += rec['IdList']
  return ids


In [ ]:
#Can be a separate function

max = 40
strain = 'Escherichia coli K-12'
# dbs = 'ipg'
query = str(strain)+'[ORGN] AND receptor[All fields]'
print(query)
#Execute the function with these parameters

ids = retrieve_ids(max, query)

Escherichia coli K-12[ORGN] AND receptor[All fields]


In [ ]:
import time
def retrieve_summary(ids, dbs, max):
  titles = []
  acc = []
  start = 0
  while start < len(ids):
    idsfrag = ids[start:start + max]
    retrieval = False

    while not retrieval:
      try:
        handle = Entrez.esummary(db=dbs, id = idsfrag, retmax = max)
        ipgsum = Entrez.read(handle)
        handle.close()
        retrieval = True
      except Exception as error:
        print('Error retrieving data, trying again in 1 second:', error)
        time.sleep(1)

    for entry in ipgsum['DocumentSummarySet']['DocumentSummary']:
      titles.append(entry['Title'])
      acc.append(entry['Accession'])


    start += max
  return titles, acc

In [ ]:
max = 40
dbs = 'ipg'
#Execute the function with these parameters
titles, acc = retrieve_summary(ids, dbs, max)

Error retrieving data, trying again in 1 second: Failed to parse the XML data (no element found: line 2, column 0). Please make sure that the input data are not corrupted.


In [ ]:
#Retrieving AA sequences

def fetch_sequences(acc):
  success = False

  while not success:
    try:
      handle = Entrez.efetch(db = 'protein', id = acc, rettype = 'gb', retmode = 'text')
      output = list(SeqIO.parse(handle, 'gb'))
      handle.close()
      success = True
    except Exception as error:
      print('Error fetching data, trying again in 1 second:', error)
      time.sleep(1)

  #Reading sequences and adding them to a list
  aaseqs = [str(entry.seq) for entry in output]
  return aaseqs


In [ ]:
#Execute the function above
aaseqs = fetch_sequences(acc)

In [ ]:
#Naming unnamed proteins
c = 1
for i in range(len(titles)):
  if titles[i] == '':
    titles[i] = 'unnamed protein v'+str(c)
    c += 1


In [ ]:
#Version 1
#Create a unique title list
#Aminoacid sequence not considered
#Comparison will be by protein name

titles_unique = []
for title in titles:
  if title not in titles_unique:
    titles_unique.append(title)

In [ ]:
#Version 1 dictionary
#If multiple AA sequences for a protein, add them to a unique key
#Unique keys, multiple values

# protein_dictionary = {}
# for key, value in zip(titles, aaseqs):
#   if key not in protein_dictionary:
#     protein_dictionary[key] = [value]
#   else:
#     protein_dictionary[key].append(value)


In [ ]:
#Version 2 dictionary
#Create an alternative list that accounts for same protein names
#Unique keys, unique values
#Comparison will be by AA seq

titles_version = []
key_counter = {}
protein_dictionary_v2 = {}

for key in titles:
  if key in key_counter.keys():
    key_counter[key] += 1
    key_title = str(key) + ' v' + str(key_counter[key])
  else:
    key_counter[key] = 1
    key_title = key
  titles_version.append(key_title)

for key, value in zip(titles_version, aaseqs):
  protein_dictionary_v2[key] = value


In [ ]:
#Creating DataFrame of Version 1 dictionary
df = pd.DataFrame(titles_unique, columns=['Protein Name'])
df[strain] = df['Protein Name'].isin(titles)

df

,Protein Name,Escherichia coli K-12
0,methyl-accepting chemotaxis protein Trg,True
1,hypothetical protein,True
2,high-affinity branched-chain amino acid ABC tr...,True
3,MFS transporter family glucose-6-phosphate rec...,True
4,bacteriophage adsorption protein NfrA,True
...,...,...
74,Hypothetical protein,True
75,tonB-dependent receptor yncD,True
76,ferrienterobactin receptor precursor,True
77,phage lambda receptor protein,True


In [ ]:
#Creating DataFrame of Version 2 dictionary
#Compared dictionary values (accumulative) with AA sequence values (unique for each query)
#Returns True/False

df2 = pd.DataFrame(protein_dictionary_v2.items(), columns=['Protein Name','AAseq'])
df2[strain] = df2['AAseq'].isin(aaseqs)
df2.drop('AAseq', axis = 1, inplace = True)

df2

,Protein Name,Escherichia coli K-12
0,methyl-accepting chemotaxis protein Trg,True
1,hypothetical protein,True
2,high-affinity branched-chain amino acid ABC tr...,True
3,MFS transporter family glucose-6-phosphate rec...,True
4,bacteriophage adsorption protein NfrA,True
...,...,...
365,TonB-dependent receptor plug domain-containing...,True
366,TonB-dependent receptor v23,True
367,ferrienterobactin receptor precursor,True
368,phage lambda receptor protein,True


In [ ]:
#Download pathogenic bacteria list from Barlett et al.
#Store it in a dataframe
url = 'https://github.com/padpadpadpad/bartlett_et_al_2022_human_pathogens/raw/master/data/bacteria_human_pathogens.xlsx'
bdf = pd.read_excel(url, sheet_name='Tab 6 Full List', usecols="F:G", skiprows=0)

In [ ]:
#Convert dataframe to list
#Join the genus and species column

pblist = list(bdf['genus'] + ' ' + bdf['species'])

In [ ]:
def retrieve_ids_nucl(max, query):
  ids = []
  start = 0
  while(True):
    try:
      handle = Entrez.esearch(db = 'nucleotide', retmax = max, retstart = start, term = query)
      rec = Entrez.read(handle)
      handle.close()

    except Exception as error:
      print('Search failed, trying again in 1 second:', error)
      time.sleep(1)
      continue

    if len(rec['IdList']) == 0:
      break

    start += max
    ids += rec['IdList']
  return ids


In [ ]:
max = 100
query = 'Viruses[ORGN] AND phage[All fields] AND srcdb_refseq[PROP] \
NOT wgs[PROP] NOT cellular organisms[ORGN] NOT AC_000001:AC_999999[PACC]'

#Execute the function with these parameters
phageids = retrieve_ids_nucl(max, query)

In [ ]:
#Input: IDs of phages
#Output: List of bacterial hosts
def phageid_to_host(phageids):
  hostlist = []
  for id in phageids:
    try:
      handle = Entrez.efetch(db="nucleotide", id=id, rettype="gb",
                            retmode="text", seq_start = 1, seq_stop = 1)
      source = SeqIO.read(handle, 'gb')
      handle.close()

      features = source.features[0]
      qual = features.qualifiers
      # qual = source.features[0].qualifiers

      for key,value in qual.items():
        if 'host' in key:
          hostlist.append(value[0])
          break

    except Exception as error:
      print('Error fetching data, trying again in 1 second:', error)
      time.sleep(1)
      continue

  return hostlist

In [ ]:
phageid_to_host(phageids[0:10])

['Aeromonas hydrophila 4.3T',
 'Aeromonas hydrophila BT09',
 'Aeromonas veronii',
 'Vibrio parahaemolyticus',
 'Escherichia coli',
 'Escherichia coli O156:H25',
 'Escherichia sp.',
 'Escherichia coli',
 'Escherichia coli',
 'Yersinia pseudotuberculosis']

In [ ]:
len(hostlist)

4878